In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import fgbuster
import pysm3
import pysm3.units as u
import pandas as pd
import emcee
import camb
from tqdm import tqdm

from scipy.optimize import minimize
import os

from fgbuster import CMB, Dust, Synchrotron, get_instrument, MixingMatrix

from getdist import MCSamples, plots

import warnings
warnings.filterwarnings('ignore')

import getdist
from getdist import MCSamples, plots

In [2]:
specifications = {
    28: (39.9, 16.5),
    35: (31.9, 13.3),
    45: (24.8, 11.9),
    65: (17.1, 8.9),
    75: (14.91, 5.1),
    95: (11.7, 4.6),
    115: (9.72, 3.1),
    130: (8.59, 3.1),
    145: (7.70, 2.4),
    165: (6.77, 2.5),
    190: (5.88, 2.8),
    220: (5.08, 3.3),
    275: (4.06, 6.3),
    340: (3.28, 11.4),
    390: (2.86, 21.9),
    450: (2.48, 43.4),
    520: (2.14, 102.0),
    600: (1.86, 288.0),
    700: (1.59, 1122.0),
    850: (1.31, 9550.0),
}

# frequency_channels = np.array([28, 65, 115, 165, 275, 450, 700])
frequency_channels = np.array([28, 35, 45, 65, 75, 95, 115, 130, 145, 165, 190, 220, 275, 340, 390, 450, 520, 600, 700, 850])
nfreqs = len(frequency_channels)

noise_rms_pol = np.array([specifications[int(frequency)][1] for frequency in frequency_channels])
noise_rms_temp = noise_rms_pol / np.sqrt(2)

conv_factor = (np.pi/(180 * 60))

noise_rms_pol = noise_rms_pol * conv_factor
noise_rms_temp = noise_rms_temp * conv_factor

In [3]:
nside = 128
nlmax = 2 * nside

# alpha = -1
# ell0 = nlmax/2
alpha = -np.arange(10, 50, 2)/10
ell0 = np.round(np.linspace(2, nlmax, nfreqs))

ells = np.arange(2,nlmax+1)
coeff = ells*(ells+1)/(2*np.pi)

In [6]:
beams = np.array([specifications[int(frequency)][0] for frequency in frequency_channels])
target_resol = 39.9*u.arcmin
target_beam = hp.gauss_beam(np.radians(target_resol.value/60), lmax= 3*nside - 1, pol=True)
target_beam_T = target_beam[:,0]
target_beam_p = target_beam[:,1]

b_f = np.array([hp.gauss_beam(np.radians(theta/60.0), lmax= 3*nside - 1, pol=True).T for theta in beams])
beam_T = b_f[:,0]
beam_pol = b_f[:,1]
ratio_T = np.array(target_beam_T/beam_T)[:,:nlmax+1]
ratio_P = (target_beam_p/beam_pol)[:,:nlmax+1]

In [7]:
foreground_config = pysm3.Sky(nside=nside, 
                    preset_strings=["d0", "s0"],  # Synchrotron ('s0') and Thermal Dust ('d0') models
                    output_unit="uK_CMB")  # Output units in microKelvin (CMB temperature units)

In [8]:
foreground_balms = np.zeros((nfreqs, hp.Alm.getsize(nlmax)), dtype = np.complex128)
for j, frequency in enumerate(frequency_channels):
    foreground_input_map = foreground_config.get_emission(frequency * u.GHz)
    foreground_balms[j] = hp.map2alm(foreground_input_map, lmax = nlmax)[2]  #only [2] which is B-mode is chosen

In [9]:
def Observation_Spectra(alms):
    Nf_local = alms.shape[0]
    alm_size = alms.shape[-1]
    nlmax = hp.Alm.getlmax(alm_size)
    
    # Allocate array for Cl; for each l, we have an n_channels x n_channels matrix.
    C = np.zeros((nlmax+1, Nf_local, Nf_local))
    
    # Loop over channel pairs.
    for f in range(Nf_local):
        for g in range(f, Nf_local):
            # Compute the cross-power spectrum between channel f and g.
            C_fg = hp.alm2cl(alms[f], alms[g], lmax = nlmax)
            C[:, f, g] = C_fg
            C[:, g, f] = C_fg  # symmetry: C_ij = C_ji.
    
    return C

In [10]:
foreground_sp = Observation_Spectra(foreground_balms)

In [11]:
def Noise_Spectra(noise_rms, alpha_knee, l_knee, nlmax):
    Nf_local = len(noise_rms)
    N = np.zeros((nlmax+1, Nf_local, Nf_local))
    noise_white = noise_rms**2 #np.power(noise_rms, 2.0)
    for l in range(2, nlmax+1):
        noise_sp = noise_white * (1 + (l / l_knee)**alpha_knee)
        N[l] = np.diag(noise_sp)
    return N

In [12]:
def get_weights(A, N):
    weights = []
    ll, mm = hp.Alm.getlm(nlmax)
    
    for ell in range(2, nlmax+1):
        N_inv_s = np.linalg.inv(N[ell])
        ANA = A.T @ N_inv_s @ A  
        AN = A.T @ N_inv_s
        weights.append((np.linalg.inv(ANA) @ AN))
           
    return np.array(weights)

In [13]:
cmb = CMB()
dust = Dust(353)
synch = Synchrotron(20)
sky_components = [cmb, dust, synch]
ncomps = len(sky_components)

MM = MixingMatrix(*sky_components)
A_evaluator = MM.evaluator(frequency_channels)

In [14]:
def get_stat_fg_residuals(samples, noise_params):
    foreground_mean = np.mean(samples, axis = 0)
    foreground_sigma = np.std(samples, axis = 0)
    distribution = np.array([np.random.normal(loc = mean, scale = std, size = 100) for mean, std in zip(foreground_mean, foreground_sigma)])

    alpha, ell0 = noise_params
    noise_sp = Noise_Spectra(noise_rms_pol, alpha_knee = alpha, l_knee = ell0, nlmax = nlmax)

    res_foreground_sp = []
    
    for i in range(distribution.shape[-1]):
        foreground_params = distribution[:, i]
        mixing_matrix = A_evaluator(foreground_params)
        weights = get_weights(mixing_matrix, noise_sp)[:,0]
        
        Wt_fg_W = np.einsum('ij,ijk,ik->i', weights, foreground_sp[2:], weights)
        res_foreground_sp.append(Wt_fg_W)

    return np.mean(res_foreground_sp, axis = 0)

In [15]:
# #using the chains to get the weights and then the statistical foreground residuals

samples_beta = np.load('results/samples_bavlv_white.npy')[:, -3:]
fg_residuals = get_stat_fg_residuals(samples_beta, [alpha, ell0])

In [16]:
np.save('post-proc/stat_fg/res_bavlv_white.npy', fg_residuals/target_beam_p[ells]**2)